In [1]:
#question 1 (b)
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_absolute_error

In [2]:
df = pd.read_csv(r"C:\srividya\analytics\weekly_sales_dataset.csv")

In [3]:
#1 Train a Baseline Model
numeric_df = df.select_dtypes(include='number').dropna()

In [7]:
X = numeric_df.drop(columns=['Weekly_Sales'])
y = numeric_df['Weekly_Sales']

In [8]:
baseline_model = LinearRegression()
baseline_model.fit(X, y)

LinearRegression()

In [9]:
y_pred = baseline_model.predict(X)

In [11]:
r2 = r2_score(y, y_pred)
mae = mean_absolute_error(y, y_pred)

In [12]:
r2, mae

(0.8495053610084965, 96.31604096238722)

In [13]:
#interpretation 
#The linear regression model explains a significant portion of weekly sales variation.
#MAE is chosen for its clear business interpretation as average sales error.
#Given its transparency and stability, the model is appropriate for budget allocation decisions.


In [14]:
#2Counterfactual Experiment
X_counterfactual = X.copy()

In [15]:
X_counterfactual['Advertising_Spend'] = X_counterfactual['Advertising_Spend'] * 1.20


In [16]:
baseline_sales = baseline_model.predict(X)
counterfactual_sales = baseline_model.predict(X_counterfactual)

In [17]:
avg_baseline = baseline_sales.mean()
avg_counterfactual = counterfactual_sales.mean()
percentage_change = ((avg_counterfactual - avg_baseline) / avg_baseline) * 100

In [18]:
avg_baseline, avg_counterfactual, percentage_change

(np.float64(848.7828333333334),
 np.float64(1038.4526804961035),
 np.float64(22.346098402803477))

In [19]:
#Interpretation
#The experiment increases advertising spend by 20% while keeping other factors unchanged.
#Sales rise accordingly, which aligns with the positive impact of advertising in the model.
#This is economically reasonable, though the linear assumption may overstate gains at higher spend levels since diminishing returns are not captured.

In [27]:
#3 Intentional Misspecification and Model Risk
X_misspecified = X.drop(columns=['Advertising_Spend'])

In [21]:
misspecified_model = LinearRegression()
misspecified_model.fit(X_misspecified, y)

LinearRegression()

In [22]:
y_miss_pred = misspecified_model.predict(X_misspecified)

In [23]:
r2_miss = r2_score(y, y_miss_pred)
mae_miss = mean_absolute_error(y, y_miss_pred)

In [24]:
r2, r2_miss, mae, mae_miss

(0.8495053610084965, 0.3489698366866034, 96.31604096238722, 205.82579660836728)

In [25]:

coeff_baseline = pd.Series(baseline_model.coef_, index=X.columns)
coeff_miss = pd.Series(misspecified_model.coef_, index=X_misspecified.columns)

coeff_baseline, coeff_miss


(Advertising_Spend     1.899227
 Price               -20.124950
 Competitor_Price     13.372099
 dtype: float64,
 Price              -21.34195
 Competitor_Price    12.11365
 dtype: float64)

In [1]:
# Interpretation
# Removing advertising spend leads to a clear drop in model performance and alters the
# coefficients of remaining variables. This indicates that the model reallocates the
# missing effect to other features, creating model risk that can result in poor budget
# allocation and potential revenue loss.


In [ ]:
# 4. Scenario Where the Model Should Not Be Deployed
# I would not deploy this model in situations where advertising effects are non-linear
# or show saturation at higher spend levels. Even if performance metrics appear strong,
# the linear assumption can overestimate returns, creating significant business risk
# through over-investment.
